In [2]:
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import json

In [3]:
def load_file(file):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [39]:
def generate_better_characters(file):
    data = load_file(file)
    print(len(data))
    new_data = []
    
    for item in data:
        new_data.append(item)
    
    for item in data:
        item = item.replace("The", "").replace("the", "").replace("And", "").replace("and", "") 
        names  = item.split(" ")
        for name in names:
            name = name.strip()
            new_data.append(name)

        if "(" in item:
            names = item.split("(")
            for name in names:
                name = name.replace("(", "").strip()
                new_data.append(name)
        
        if "," in item:
            names = item.split(",")
            for name in names:
                name = name.replace("and", "").strip()
                if " " in name:
                    new_names = name.split()
                    for x in new_names:
                        x = x.strip()
                        new_data.append(x)
                new_data.append(name)
    print(len(new_data))
    final_data = []
    titles = ["Dr.", "Professor", "Mr.", "Mrs.", "Ms.", "Miss", "Aunt", "Uncle", "Mr. and Mrs."]

    for character in new_data:
        if "" != character:
            final_data.append(character)
            for title in titles:
                titled_char = f"{title} {character}"
                final_data.append(titled_char)
    print(len(final_data))

    final_data = list(set(final_data))
    print(len(final_data))
    final_data.sort()
    return final_data

In [40]:
def create_training_data(file, type):
    data = generate_better_characters(file)
    patterns = []
    for item in data:
        pattern = {
            "label": type,
            "pattern": item
        }
        patterns.append(pattern)
    return patterns

In [43]:
def generate_rules(patterns):
    nlp = English()
    ruler = EntityRuler(nlp)
    ruler.add_patterns(patterns)
    nlp.add_pipe("entity_ruler", last=True)
    nlp.to_disk("./hp_ner")

In [44]:
hp_chars = generate_better_characters('./data/hp_characters.json')
patterns = create_training_data("./data/hp_characters.json", "PERSON")
generate_rules(patterns)

207
622
6160
5119
207
622
6160
5119
